# UoA Copcsi 765 Assignemnt 1
Shuo Mao 

In [1]:
# loading data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

file_path = 'website-phishing.csv'
data = pd.read_csv(file_path)


# Splitting the dataset into features (X) and target (y)
X = data.iloc[:, :-1].values# Selecting all expect y(class)
y = data.iloc[:, -1].values# Selecting Class

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)


# 1: Decision Stump
A decision stump is a decision tree with a depth of 1. It splits the dataset using the best feature based on a criterion like Gini impurity or information gain.

In [29]:
import numpy as np

class DecisionStump:
    def __init__(self):
        # Initialize the decision stump with default values
        self.threshold = None  # The value to decide if a data point goes left or right
        self.feature_index = None  # The index of the feature to split on
        self.left_value = None  # The most common class on the left side of the stump
        self.right_value = None  # The most common class on the right side of the stump
    
    
    def fit(self, X, y):
        best_info_gain = -1  # Initialize the best information gain to a value that will be updated
        
        # Iterate over all features in the dataset
        for feature_index in range(X.shape[1]): # in this case feature_index repersending columns in row dataset
            thresholds = np.unique(X[:, feature_index])  # Get all unique values of the feature as potential thresholds
            print(thresholds , " : thresholds")
            for threshold in thresholds:
                print(threshold , " :threshold")
                # Calculate the information gain of splitting on this feature at this threshold
                info_gain = self.information_gain(y, X[:, feature_index], threshold)
                print(info_gain, " : info gain")
                if info_gain > best_info_gain:
                    # If this split provides a better information gain, update the stump's parameters
                    best_info_gain = info_gain
                    self.threshold = threshold
                    self.feature_index = feature_index
        
        # After finding the best split, determine the output value (class) for each side of the split
        # This uses the mean of the target values, which implicitly handles binary classification (0 and 1)
        self.left_value = np.round(np.mean(y[X[:, self.feature_index] <= self.threshold]))
        self.right_value = np.round(np.mean(y[X[:, self.feature_index] > self.threshold]))
        
        # Determine the value for the left and right child nodes
        self.left_value = np.round(np.mean(y[X[:, self.feature_index] <= self.threshold]))
        self.right_value = np.round(np.mean(y[X[:, self.feature_index] > self.threshold]))
        
    def predict(self, X):
        # Generate predictions based on the threshold and feature index
        predictions = np.where(X[:, self.feature_index] <= self.threshold, self.left_value, self.right_value)
        return predictions

    
    def information_gain(self, y, feature_column, threshold):
        parent_entropy = self.entropy(y)  # Calculate the entropy before the split
        
        # Create masks to split the data based on the threshold
        left_child_index = feature_column <= threshold
        right_child_index = feature_column > threshold
        
        # Ensure both sides of the split contain samples
        if np.any(left_child_index) and np.any(right_child_index):
            n = len(y)
            n_left = np.sum(left_child_index)  # Number of samples in the left split
            n_right = np.sum(right_child_index)  # Number of samples in the right split
            # Weighted average of child entropies
            child_entropy = (n_left / n) * self.entropy(y[left_child_index]) + (n_right / n) * self.entropy(y[right_child_index])
            info_gain = parent_entropy - child_entropy  # Information gain is reduction in entropy
            return info_gain
        else:
            return 0

    
    def entropy(self, y):
        if np.any(y < 0):  # Handle datasets where the negative class is labeled as -1
            y = np.where(y == -1, 0, 1)
        proportions = np.bincount(y) / len(y)
        return -np.sum([p * np.log2(p) for p in proportions if p > 0])



In [ ]:
# Assume the dataset has been loaded into a DataFrame named data
# and the necessary preprocessing has been done.



# Initialize and fit the decision stump
stump = DecisionStump()
stump.fit(X_train, y_train)

# Use the stump to make predictions
predictions = stump.predict(X_test)
# Optionally, evaluate the stump's performance
accuracy = np.mean(predictions == y_test)

print(f'Accuracy of the decision stump: {accuracy:.4f}')


# 2: Unpruned Decision Tree
An unpruned decision tree grows until all leaves are pure or until every leaf contains a minimum number of samples. Here, you recursively split the nodes based on the best split criterion.

In [11]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.threshold = None
        self.feature_index = None
        self.left = None
        self.right = None
        self.value = None
        self.depth = 0

    def fit(self, X, y, depth=0):
        self.depth = depth
        if len(np.unique(y)) == 1 or self.depth == self.max_depth:
            self.value = self._most_common_label(y)
            return
        
        best_info_gain = -1
        for feature_index in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                info_gain = self._information_gain(y, X[:, feature_index], threshold)
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    self.threshold = threshold
                    self.feature_index = feature_index
                    
        if best_info_gain == 0:
            self.value = self._most_common_label(y)
            return
        
        left_indexes = X[:, self.feature_index] <= self.threshold
        X_left, y_left = X[left_indexes], y[left_indexes]
        X_right, y_right = X[~left_indexes], y[~left_indexes]
        
        self.left = DecisionTree(self.max_depth)
        self.left.fit(X_left, y_left, depth + 1)
        self.right = DecisionTree(self.max_depth)
        self.right.fit(X_right, y_right, depth + 1)

    def predict(self, X):
        return np.array([self._predict(inputs) for inputs in X])
    
    def _predict(self, inputs):
        if self.value is not None:
            return self.value
        if inputs[self.feature_index] <= self.threshold:
            return self.left._predict(inputs)
        return self.right._predict(inputs)

    def _information_gain(self, y, feature_column, threshold):
        parent_entropy = self.entropy(y)  # Calculate the entropy before the split
        
        # Create masks to split the data based on the threshold
        left_child_index = feature_column <= threshold
        right_child_index = feature_column > threshold
        
        # Ensure both sides of the split contain samples
        if np.any(left_child_index) and np.any(right_child_index):
            n = len(y)
            n_left = np.sum(left_child_index)  # Number of samples in the left split
            n_right = np.sum(right_child_index)  # Number of samples in the right split
            # Weighted average of child entropies
            child_entropy = (n_left / n) * self.entropy(y[left_child_index]) + (n_right / n) * self.entropy(y[right_child_index])
            info_gain = parent_entropy - child_entropy  # Information gain is reduction in entropy
            return info_gain
        else:
            return 0
    
    def entropy(self, y):
        if np.any(y < 0):  # Handle datasets where the negative class is labeled as -1
            y = np.where(y == -1, 0, 1)
        proportions = np.bincount(y) / len(y)
        return -np.sum([p * np.log2(p) for p in proportions if p > 0])

    
    def _most_common_label(self, y):
    # Adjust labels if necessary to ensure they are non-negative
        if np.min(y) < 0:
            y_adjusted = y - np.min(y)  # Shift values to make all elements non-negative
        else:
            y_adjusted = y
        return np.bincount(y_adjusted).argmax() + np.min(y)  # Add the minimum value back to the result




In [12]:
# Assuming you've already loaded and prepared your dataset into X and y

# Splitting the dataset into features (X) and target (y)
X = data.iloc[:, :-1].values# Selecting all expect y(class)
y = data.iloc[:, -1].values# Selecting Class

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=4)

tree = DecisionTree(max_depth=5)  # Example: limit the depth to prevent overfitting
tree.fit(X_train, y_train)

# Predictions and evaluation
predictions = tree.predict(X_test)
accuracy = np.mean(predictions == y_test)
print(accuracy)
print(f'Accuracy: {accuracy:.4f}')


0.6955499276410999
Accuracy: 0.6955


# 3: Pruned Decision Tree
Pruning can be done using several strategies like reduced error pruning (post-pruning) or setting a maximum depth (pre-pruning). In a Jupyter notebook, you can extend the unpruned decision tree with pruning capabilities.

In [16]:
import numpy as np

class PrunedDecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.threshold = None
        self.feature_index = None
        self.left = None
        self.right = None
        self.value = None
        self.depth = 0
        self.alpha = 0

    def fit(self, X, y, depth=0):
        self.depth = depth
        if len(np.unique(y)) == 1 or self.depth == self.max_depth:
            self.value = self._most_common_label(y)
            return
        
        best_info_gain = -1
        for feature_index in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                info_gain = self._information_gain(y, X[:, feature_index], threshold)
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    self.threshold = threshold
                    self.feature_index = feature_index
                    
        if best_info_gain == 0:
            self.value = self._most_common_label(y)
            return
        
        left_indexes = X[:, self.feature_index] <= self.threshold
        X_left, y_left = X[left_indexes], y[left_indexes]
        X_right, y_right = X[~left_indexes], y[~left_indexes]
        
        self.left = PrunedDecisionTree(self.max_depth)
        self.left.fit(X_left, y_left, depth + 1)
        self.right = PrunedDecisionTree(self.max_depth)
        self.right.fit(X_right, y_right, depth + 1)

    def predict(self, X):
        return np.array([self._predict(inputs) for inputs in X])
    
    def _predict(self, inputs):
        if self.value is not None:
            return self.value
        if inputs[self.feature_index] <= self.threshold:
            return self.left._predict(inputs)
        return self.right._predict(inputs)

    def _information_gain(self, y, feature_column, threshold):
        parent_entropy = self.entropy(y)  # Calculate the entropy before the split
        
        # Create masks to split the data based on the threshold
        left_child_index = feature_column <= threshold
        right_child_index = feature_column > threshold
        
        # Ensure both sides of the split contain samples
        if np.any(left_child_index) and np.any(right_child_index):
            n = len(y)
            n_left = np.sum(left_child_index)  # Number of samples in the left split
            n_right = np.sum(right_child_index)  # Number of samples in the right split
            # Weighted average of child entropies
            child_entropy = (n_left / n) * self.entropy(y[left_child_index]) + (n_right / n) * self.entropy(y[right_child_index])
            info_gain = parent_entropy - child_entropy  # Information gain is reduction in entropy
            return info_gain
        else:
            return 0
    
    def entropy(self, y):
        if np.any(y < 0):  # Handle datasets where the negative class is labeled as -1
            y = np.where(y == -1, 0, 1)
        proportions = np.bincount(y) / len(y)
        return -np.sum([p * np.log2(p) for p in proportions if p > 0])

    
    def _most_common_label(self, y):
    # Adjust labels if necessary to ensure they are non-negative
        if np.min(y) < 0:
            y_adjusted = y - np.min(y)  # Shift values to make all elements non-negative
        else:
            y_adjusted = y
        return np.bincount(y_adjusted).argmax() + np.min(y)  # Add the minimum value back to the result


    def prune(self, X_val, y_val):
        """
        Prune the decision tree using cost-complexity pruning and a validation set.
        """
        # Check if the current node is a leaf
        if self.is_leaf():
            return
        
        # If the current node has children, recursively prune them
        if self.left is not None:
            self.left.prune(X_val, y_val)
        if self.right is not None:
            self.right.prune(X_val, y_val)
        
        # Check if pruning the current node reduces the cost on the validation set
        if not self.is_leaf():
            error_before_pruning = self._calc_validation_error(X_val, y_val)
            self._make_leaf()  # Temporarily make the current node a leaf
            error_after_pruning = self._calc_validation_error(X_val, y_val)
            
            # If pruning increases the validation error, revert the pruning
            if error_after_pruning > error_before_pruning:
                self._unmake_leaf()  # Method to revert the current node from a leaf back to its original structure

    def _calc_validation_error(self, X_val, y_val):
        """
        Calculate the error of the tree on the validation set.
        """
        predictions = self.predict(X_val)
        error = np.mean(predictions != y_val)
        return error
    
    def is_leaf(self):
        """
        Check if the current node is a leaf.
        """
        return self.left is None and self.right is None
    
    def _make_leaf(self):
        """
        Make the current node a leaf.
        """
        # Store current subtrees and value before making changes
        self.temp_left = self.left
        self.temp_right = self.right
        # You could also store self.value if necessary
        # For simplicity, we recalculate it during unmake_leaf if needed
        
        self.value = self._most_common_label(y)  # Assuming self.y stores the labels reaching this node
        self.left = None
        self.right = None

    def _unmake_leaf(self):
        """
        Revert the current node from a leaf back to its original structure.
        This method restores the child branches and the node's value.
        """
        # Check if we have stored subtrees to revert back to
        if hasattr(self, 'temp_left') and hasattr(self, 'temp_right'):
            self.left = self.temp_left
            self.right = self.temp_right
            # Restore self.value if it was stored or recalculate it if needed
            # If you're recalculating, make sure you have the necessary data available
            
            # Clean up by removing the temporary attributes
            del self.temp_left
            del self.temp_right
    

In [26]:
# Split into training+validation and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_temp.shape)
# Split training+validation set into separate training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2
# Assuming the PrunedDecisionTree class is defined as above
tree1 = PrunedDecisionTree(max_depth=2)  # Adjust max_depth as needed
print(X_train.shape, X_val.shape)
# Fit the model to the training set
# Note: Pruning is integrated into the fit method and uses the validation set
tree1.fit(X_train, y_train)
tree1.prune(X_val, y_val)
# Use the model to make predictions on the test set
predictions = tree1.predict(X_train)

# Calculate the accuracy of the model
accuracy = np.mean(predictions == y_train)
print(f'Test accuracy: {accuracy:.4f}')

(8844, 30)
(6633, 30) (2211, 30)
Test accuracy: 0.5577


In [30]:
file_path = 'bcp.csv'
data = pd.read_csv(file_path)
# Splitting the dataset into features (X) and target (y)
X = data.iloc[:, :-1].values# Selecting all expect y(class)
y = data.iloc[:, -1].values# Selecting Class
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)
# Initialize and fit the decision stump
stump = DecisionStump()
stump.fit(X_train, y_train)

# Use the stump to make predictions
predictions = stump.predict(X_test)
# Optionally, evaluate the stump's performance
accuracy = np.mean(predictions == y_test)

print(f'Accuracy of the decision stump: {accuracy:.4f}')


tree = DecisionTree(max_depth=5)  # Example: limit the depth to prevent overfitting
tree.fit(X_train, y_train)

# Predictions and evaluation
predictions = tree.predict(X_test)
accuracy = np.mean(predictions == y_test)
print(accuracy)
print(f'Accuracy: {accuracy:.4f}')


[  63375   76389   95719  128059  142932  144888  145447  160296  183913
  183936  188336  191250  242970  255644  263538  274137  303213  320675
  324382  342245  353098  369565  378275  385103  390840  411453  412300
  428598  428903  431495  434518  456282  466906  474162  486662  492268
  492561  521441  527337  527363  529329  534555  535331  536708  543558
  555977  557583  558538  560680  566346  566509  603148  608157  635844
  636130  636375  640712  653777  654546  657753  666090  666942  667204
  672113  677910  684955  688033  690557  691628  693702  695091  704097
  706426  709287  714039  721482  730881  733823  734111  740492  749653
  752904  756136  760001  760239  763235  764974  769612  776715  785615
  787451  792744  797327  798429  805448  806423  807657  809912  810104
  814265  814911  822829  824249  827627  831268  837082  837480  841769
  846423  846832  850831  855524  857774  859164  859350  866325  867392
  871549  873549  877291  877943  878358  888169  8